In [1]:
import nltk
import random
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
import json
import pickle
from sklearn.metrics import accuracy_score, f1_score

In [2]:
label_map={'neg':0 ,'sli_neg':1 ,'neutral':2 ,'sli_pos':3 ,'pos':4}

In [3]:
def load_dataset():  
    dataset=[]
    for root, dirs, files in os.walk('../Dataset'):
        for directory in dirs:
            for root_f, dirs_f, files_f in os.walk(os.path.join(root,directory)):
                for each_file in files_f:
                    with open(os.path.join(root,directory)+'/'+each_file) as f:
                        data_temp=json.load(f)    
                    dataset+=[(sentence,label_map[each_file]) for sentence in data_temp]
    return dataset

In [4]:
def get_words(x):    #returns all the words in the  data set
    vocab = []
    for sentence,polarity in x:  
        vocab.extend(sentence.split())
    return vocab

In [5]:
#execution
labeled_data=load_dataset()    #loading labeled sentences
random.shuffle(labeled_data)
#vocab=list(set(get_words(labeled_data))) #getting words 
#with open('vocab_unigram','w') as f:
#    json.dump(vocab,f)
with open('vocab_unigram') as f:
    vocab=json.load(f)

In [6]:
len(vocab)

6544

In [7]:
#bigrams
print labeled_data[:5]

[(u'everi day new day', 3), (u'work environ team will work', 3), (u'person life career', 2), (u'work hard achiev even smallest increment progress', 1), (u'also got good experi manag', 4)]


In [8]:
print len(labeled_data)

201378


In [41]:
vectorizer = CountVectorizer(vocabulary=vocab,min_df=1)
corpus=[sentence for sentence,pol in labeled_data]
features=vectorizer.fit_transform(corpus[:80000])
labels=[label for sentence,label in labeled_data[:80000]]

In [42]:
classifier=svm.SVC(kernel='linear',probability=True)

In [43]:
classifier.fit(features[:50000],labels[:50000])
#with open('svc_unigram_model','w') as f:
#    pickle.dump(classifier,f) 

In [150]:
len(vocab)

6544

In [12]:
#accuracy
with open('svc_unigram_model') as f:
    classifier=pickle.load(f)
features_test=vectorizer.fit_transform(corpus[45000:60000])
predicted = classifier.predict(features_test)
accuracy=accuracy_score(labels[45000:60000],predicted)
#with open('accuracy_svc_unigram','w') as f:                #compute accuracy and store it
#    json.dump(accuracy,f) 

In [13]:
with open('accuracy_svc_unigram') as f:
    accuracy=json.load(f)
print accuracy    

0.874066666667


In [58]:
#testing
# loading classifier
with open('svc_unigram_model') as f:
    classifier=pickle.load(f)
feature_test=[]
print 'sample :'+corpus[50001]
# creating feature set
for word in vocab: 
    if word in corpus[50005]:
        feature_test.append(1)
    else:
        feature_test.append(0)
#predicting class        
classifier.predict(feature_test)

sample :job made realiz good job


/home/pranav/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([3])

In [60]:
classifier.predict_proba(feature_test)[0][3]

/home/pranav/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


0.93479724976458467